# Adapted from https://www.youtube.com/watch?v=t1MrzuAUdoE

In [4]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import skimage.draw
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd

# Root directory of the project
ROOT_DIR = os.path.abspath("../Final_Project")

# Use leekunhee version to be compatible with tensorflow 2.x (https://github.com/leekunhee/Mask_RCNN)
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN

%matplotlib inline

In [5]:
# Check the directory paths
print("Root Directory:", ROOT_DIR)
print("Weights Path:", COCO_WEIGHTS_PATH)
print("Logs Path:", DEFAULT_LOGS_DIR)

Root Directory: /Users/tiffanyshih/Documents/Coursework/MIDS_207/Final_Project
Weights Path: /Users/tiffanyshih/Documents/Coursework/MIDS_207/Final_Project/mask_rcnn_coco.h5
Logs Path: /Users/tiffanyshih/Documents/Coursework/MIDS_207/Final_Project/logs


In [6]:
# REQUIRED: Load labels json file containing image object labels
# Note: This will take awhile to load

with open('train_clean/bdd100k_labels_images_train.json') as json_file:
    train_labels = json.load(json_file)

with open('val_clean/bdd100k_labels_images_val.json') as json_file:
    test_labels = json.load(json_file)
    
#with open('val_clean/bdd100k_labels_images_test.json') as json_file:
#    test_labels = json.load(json_file)

In [7]:
# REQUIRED: Normalize semi-structured JSON data into a pd dataframe
train_labels_normalized = pd.json_normalize(train_labels)
test_labels_normalized = pd.json_normalize(test_labels)

In [8]:
# REQUIRED: Function used to clean data, extract labels, and extract data for sunny, 
# highway, and daytime conditions

def clean_data(labels_normalized):
    # Initialize lists of object types. Each list will contain 69863 values, each of which corresponds to an image specified as a row in data_cleaned 
    num_objects = []
    num_road = []
    num_sidewalk = []
    num_building = []
    num_wall = []
    num_fence = []
    num_pole = []
    num_traffic_light = []
    num_traffic_sign = []
    num_vegetation = []
    num_terrain = []
    num_sky = []
    num_person = []
    num_rider = []
    num_car = []
    num_truck = []
    num_bus = []
    num_train = []
    num_motorcycle = []
    num_bicycle = []
    num_lanes = []
    num_drivable_area = []

    # Iterate through each image in train_labels_normalized, sum up the number of objects corresponding to each object type, and append to
    # the corresponding object list
    for index, row in labels_normalized.iterrows():
        num_objects.append(len(row['labels']))
        num_road.append(sum(x['category'] == 'road' for x in row['labels']))
        num_sidewalk.append(sum(x['category'] == 'sidewalk' for x in row['labels']))
        num_building.append(sum(x['category'] == 'building' for x in row['labels']))
        num_wall.append(sum(x['category'] == 'wall' for x in row['labels']))
        num_fence.append(sum(x['category'] == 'fence' for x in row['labels']))
        num_pole.append(sum(x['category'] == 'pole' for x in row['labels']))
        num_traffic_light.append(sum(x['category'] == 'traffic light' for x in row['labels']))
        num_traffic_sign.append(sum(x['category'] == 'traffic sign' for x in row['labels']))
        num_vegetation.append(sum(x['category'] == 'vegetation' for x in row['labels']))
        num_terrain.append(sum(x['category'] == 'terrain' for x in row['labels']))
        num_sky.append(sum(x['category'] == 'sky' for x in row['labels']))
        num_person.append(sum(x['category'] == 'person' for x in row['labels']))
        num_rider.append(sum(x['category'] == 'rider' for x in row['labels']))
        num_car.append(sum(x['category'] == 'car' for x in row['labels']))
        num_truck.append(sum(x['category'] == 'truck' for x in row['labels']))
        num_bus.append(sum(x['category'] == 'bus' for x in row['labels']))
        num_train.append(sum(x['category'] == 'train' for x in row['labels']))
        num_motorcycle.append(sum(x['category'] == 'motorcycle' for x in row['labels']))
        num_bicycle.append(sum(x['category'] == 'bicycle' for x in row['labels']))
        num_lanes.append(sum(x['category'] == 'lane' for x in row['labels']))
        num_drivable_area.append(sum(x['category'] == 'drivable area' for x in row['labels']))

    labels_normalized['num_objects'] = num_objects
    labels_normalized['num_sidewalk'] = num_sidewalk
    labels_normalized['num_building'] = num_building
    labels_normalized['num_wall'] = num_wall
    labels_normalized['num_fence'] = num_fence
    labels_normalized['num_pole'] = num_pole
    labels_normalized['num_traffic_light'] = num_traffic_light
    labels_normalized['num_traffic_sign'] = num_traffic_sign
    labels_normalized['num_vegetation'] = num_vegetation
    labels_normalized['num_terrain'] = num_terrain
    labels_normalized['num_sky'] = num_sky
    labels_normalized['num_person'] = num_person
    labels_normalized['num_rider'] = num_rider
    labels_normalized['num_car'] = num_car
    labels_normalized['num_truck'] = num_truck
    labels_normalized['num_bus'] = num_bus
    labels_normalized['num_train'] = num_train
    labels_normalized['num_motorcycle'] = num_motorcycle
    labels_normalized['num_bicycle'] = num_bicycle
    labels_normalized['num_lanes'] = num_lanes
    labels_normalized['num_drivable_area'] = num_drivable_area

    # Filter only for images where weather == "clear", scene == "highway", and timeofday == "daytime". 
    # Note: A better implementation may be to filter the dataframe prior to counting the objects per image, 
    # but it may be useful to keep the train_labels_normalized and object counts for each image in case we want to use different filtering values.

    labels_cleaned = labels_normalized[(labels_normalized['attributes.weather'] == "clear") & 
                   (labels_normalized['attributes.scene'] == "highway") & 
                   (labels_normalized['attributes.timeofday'] == "daytime")]
    return labels_cleaned

# Clean Training & Validation Labels
train_labels_cleaned = clean_data(train_labels_normalized)

# Clean Test Labels
test_labels_cleaned = clean_data(test_labels_normalized)

# Split training dataset to training and validation
train_labels_split = train_labels_cleaned.sample(frac = 0.9)
val_labels_split = train_labels_cleaned.drop(train_labels_split.index)

In [ ]:
# Not sure if this is needed yet
def flatten_json(y):
        out = {}

        def flatten(x, name=''):
            if type(x) is dict:
                for a in x:
                    flatten(x[a], name + a + '_')
            elif type(x) is list:
                i = 0
                for a in x:
                    flatten(a, name + str(i) + '_')
                    i += 1
            else:
                out[name[:-1]] = x

        flatten(y)
        return out

In [12]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + drivable area

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [17]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        """Load a subset of the driving dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        # Add classes. We have only one class to add.
        self.add_class("object", 1, "drivable area")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset + "_clean")

        # Given an image (i.e. a row in train_labels_split)
        # Get all the items that are considered drivable area
        # For each item, save the x and y vertices into two lists

        for index, image in train_labels_split.iterrows():

            polygons = []
            objects = []
            for obj in image['labels']:
                x = []
                y = []
                if obj['category'] == 'drivable area':
                    for coord in obj['poly2d'][0]['vertices']:
                        x.append(coord[0])
                        y.append(coord[1])
                    polygons.append({'name': 'polygon', 
                                      'all_points_x': x, 
                                      'all_points_y': y})
                    objects.append('drivable area')

            image_path = os.path.join(dataset_dir, image['name'])
            
            # Only use commented code if have more than 1 object. Otherwise use the uncommented one that sets 1 to all objects
            # name_dict = {"Car": 1,"Truck": 2}
            # num_ids = [name_dict[a] for a in objects]
            num_ids = [1 for a in objects]
            
            # Don't need to do these steps because all images are the same height and width
            # img = skimage.io.imread(image_path)
            # height, width = img.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=image['name'],  # use file name as a unique image id
                path=image_path,
                width=1280, height=720,
                polygons=polygons,
                num_ids=num_ids
                )
        return

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)
    
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [18]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(ROOT_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("ROOT_DIR", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=20,
                layers='heads')


In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /Users/tiffanyshih/Documents/Coursework/MIDS_207/Final_Project/logs/object20220401T1447/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed